# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict

In [2]:
# Configuration and paths
PUBMED_TARGETS_DIR = 'inputs/pubmed-targets'
DRUGBANK_TABLES_DIR = 'inputs/drugbank-tables'
MAPPING_FILE = 'inputs/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'passage_output.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]
client = OpenAI()


In [3]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [4]:
def get_relevant_table_content(tables, table_names, max_rows=5):
    """Extract relevant content from tables for context"""
    print("Debug - Available tables:", tables.keys())
    print("Debug - Looking for table_names:", table_names)

    table_content = {}
    for table_name in table_names:
        # Remove .csv extension if present
        base_table_name = table_name.replace('.csv', '')
        
        if base_table_name in tables:
            df = tables[base_table_name]
            table_content[base_table_name] = {
                'columns': list(df.columns),
                'sample': df.head(max_rows).to_dict('records')
            }
        else:
            print(f"Debug - Table '{base_table_name}' not found in available tables")
    return table_content

def generate_questions_for_passage(passage_id, passage_text, tables, relevant_table_names, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""

    # print("tables", tables)
    # print("relevant_table_names", relevant_table_names)
    
    # Limit passage length if too long (e.g., first 1000 characters)
    passage_text = passage_text[:1000] + "..." if len(passage_text) > 1000 else passage_text
    
    # Limit to maximum 3 relevant tables
    relevant_table_names = relevant_table_names[:3]
    
    # Get relevant table content
    table_content = get_relevant_table_content(tables, relevant_table_names)

    
    prompt = f"""
    Given the following passage and related tables, generate 3-5 meaningful question-answer pairs.
    IMPORTANT: Each question MUST be answerable using information from ONLY the passage.
    Only generate questions that require information from the passage.
    Focus on pharmaceutical and medical aspects.
    
    
    Passage (ID: {passage_id}):
    {passage_text}
    
    Related Tables:
    {json.dumps(table_content, indent=2)}
    
    Generate questions in the following format:
    1. question: [specific question about drug/treatment]
       answer: [detailed answer combining information from passage and tables]
       text: [passage ID if information from passage was used (e.g. Target-123456789)]
       table: [None]
    
    Ensure every question uses information from both the passage and at least one table.
    """
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content, passage_id)
    return qa_pairs

def parse_llm_response(response_text, passage_id):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': passage_id,
            'table': 'None'  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [5]:
def main():
    # Load mappings and data
    print("Loading passage-table mappings...")
    passage_table_mapping = load_passage_table_mapping()
    
    print("Loading target passages...")
    passages = load_target_passages()
    
    print("Loading DrugBank tables...")
    tables = load_drugbank_tables()
    
    # Initialize output list
    qa_pairs = []
    
    # Process each passage with its relevant tables
    for passage_id, passage_text in tqdm(list(passages.items())[:150]):
        if passage_id in passage_table_mapping:
            relevant_tables = passage_table_mapping[passage_id]
            # print(passage_id, passage_text, tables, relevant_tables)
            
            # Generate QA pairs using the passage and its relevant tables
            new_qa_pairs = generate_questions_for_passage(
                passage_id,
                passage_text,
                tables,
                relevant_tables
            )
            # new_qa_pairs = []
            print(new_qa_pairs)
            qa_pairs.extend(new_qa_pairs)
        else:
            print(f"No table mapping found for passage {passage_id}")
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [6]:
if __name__ == "__main__":
    main()

Loading passage-table mappings...
Loading target passages...
Loading DrugBank tables...


/tmp/ipykernel_35722/2028544267.py:30: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tables[table_name] = pd.read_csv(file_path)
  0%|          | 0/150 [00:00<?, ?it/s]

Debug - Available tables: dict_keys(['drugbank-drug_trans_links', 'drugbank-enzymes_polypeptides_ext_id', 'drugbank-drug_manufacturers', 'drugbank-drug_sequences', 'drugbank-drug_external_links', 'drugbank-drug_international_brands', 'drugbank-drug_mixtures', 'drugbank-drug_packagers', 'drugbank-drug_reactions_enzymes', 'drugbank-drug_enzymes_articles', 'drugbank-targets', 'drugbank-drug_carriers_textbooks', 'drugbank-drug_trans_textbooks', 'drugbank-carriers_polypeptides_ext_id', 'drugbank-drugs_attachments', 'drugbank-drug_carriers_articles', 'drugbank-enzymes_polypeptides', 'drugbank-drug_trans_articles', 'drugbank-drug_experimental_properties', 'drugbank-transporters_polypeptides_ext_id', 'drugbank-drug_dosages', 'drugbank-drug_syn', 'drugbank-drug_targ_textbooks', 'drugbank-drug_enzymes_attachments', 'drugbank-drug_carriers_links', 'drugbank-drug_salts', 'drugbank-drugs_articles', 'drugbank-targets_actions', 'drugbank-drug_patents', 'drugbank-drug_affected_organisms', 'drugbank-qu

  1%|          | 1/150 [00:04<11:18,  4.55s/it]

1. question: What is the primary clinical use of Flumazenil, and what is its preferred route of administration?
   answer: Flumazenil is primarily used to reverse the effects of benzodiazepine sedation. It has a low bioavailability when administered orally, so the preferred route of administration is intravenous.
   text: Target-8693922
   table: None

2. question: How does Flumazenil interact with benzodiazepines when administered before or with them?
   answer: When Flumazenil is administered before or with other benzodiazepines, it modifies their effects. The extent of this modification depends on factors such as the dose, duration of effect, and the relative receptor affinity of the agonist.
   text: Target-8693922
   table: None

3. question: What are the indications for using Flumazenil in a clinical setting?
   answer: The indications for using Flumazenil include the reversal of benzodiazepine-induced sedation, termination of benzodiazepine-induced anesthesia, return of spontane

...

1. question: What is the receptor profile of rotigotine, and what are its affinities for dopamine receptors?
   answer: Rotigotine, a non-ergoline dopamine agonist used for Parkinson's disease, has the highest affinity for the dopamine D3 receptor with a Ki of 0.71 nM. Its affinities for other dopamine receptors are as follows: D4.2 (3.9 nM), D4.7 (5.9 nM), D5 (5.4 nM), D2 (13.5 nM), D4.4 (15 nM), and D1 (83 nM). Additionally, it shows significant affinities for alpha-adrenergic (alpha2B, Ki=27 nM) and serotonin receptors (5-HT1A, Ki=30 nM).
   text: Target-18704368
   table: None

2. question: How does rotigotine function as an agonist in comparison to dopamine at dopamine receptors?
   answer: Rotigotine acts as a full agonist at dopamine receptors, with a higher potency compared to dopamine. Its rank order of activity is D3>D2L>D1=D5>D4.4, demonstrating potencies 2,600 and 53 times higher than dopamine at dopamine D3 and D2L receptors, respectively.
   text: Target-18704368
   table

 99%|█████████▉| 149/150 [12:13<00:05,  5.78s/it]

1. question: What is the role of Etoposide (VP-16) as a chemotherapeutic agent, and how does its combination with other drugs affect its efficacy?
   answer: Etoposide (VP-16) is a topoisomerase-II (topo II) inhibitor used in chemotherapy. Studies have shown that when VP-16 is combined with other drugs, it demonstrates better clinical responses. Specifically, the combination of moxifloxacin (MXF) and VP-16 enhances cellular topo II inhibitory activity and reduces band depletion of topo II, which suggests an increased efficacy in drug-treated cells.
   text: Target-20596674
   table: None

2. question: How does the combination of moxifloxacin (MXF) and Etoposide (VP-16) affect the cell cycle and apoptosis in HT-29 cells?
   answer: The combination of moxifloxacin (MXF) and Etoposide (VP-16) enhances cell cycle arrest at the G2/M phase and increases the appearance of the subG1 peak, which is associated with increased apoptosis in HT-29 cells. This indicates that the combination has a mor

100%|██████████| 150/150 [12:20<00:00,  4.93s/it]

1. question: What is the function of apomorphine as described in the passage, and how does it relate to dopamine receptors?
   answer: Apomorphine is a nonselective dopamine D1/D2 receptor agonist used to treat symptoms resulting from dopaminergic degeneration associated with Parkinson's disease. It functions by rapidly and reversibly increasing vesicular dopamine uptake in nerve terminals, a process that is blocked by a dopamine D2 receptor antagonist, eticlopride.
   text: Target-15178358
   table: None

2. question: How does apomorphine affect vesicular monoamine transporter-2 (VMAT-2) function according to the passage?
   answer: Apomorphine increases vesicular dopamine uptake by causing a redistribution of VMAT-2 protein within nerve terminals, which suggests its potential neuroprotective effects in models of dopaminergic degeneration.
   text: Target-15178358
   table: None

3. question: What experimental evidence is provided in the passage regarding the action of apomorphine on 